In [1]:
from warnings import filterwarnings
import os
import glob
from random import choice
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
from IPython.display import clear_output
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as img
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, utils , models
from pathlib import Path
import cv2
filterwarnings('ignore')
%matplotlib inline

In [2]:
repo_path = Path(os.getcwd()) / 'covid-chestxray-dataset'

In [3]:
import subprocess
if not repo_path.is_dir():
    process = subprocess.Popen(['git',
                                'clone',
                                'https://github.com/ieee8023/covid-chestxray-dataset.git'],
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE)
    print(process.stdout.read())
else:
    print("Data is Already There")

Data is Already There


In [4]:
pd.set_option('display.max_columns', None)

In [5]:
metadata = pd.read_csv(repo_path / 'metadata.csv')

In [6]:
x_ray_data = metadata[metadata.modality == 'X-ray']
x_ray = x_ray_data[x_ray_data.view=='PA']
final_data = x_ray[['sex','age','finding','filename','clinical_notes']]

In [7]:
final_data.head(1)

,sex,age,finding,filename,clinical_notes
0,M,65.0,COVID-19,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,infiltrate in the upper lobe of the left lung


In [8]:
illness = final_data.finding.unique()
final_data.groupby('finding').count()

,sex,age,filename,clinical_notes
finding,,,,
ARDS,3,3,4,4
COVID-19,124,114,144,141
"COVID-19, ARDS",11,11,11,11
Chlamydophila,1,1,1,1
Klebsiella,1,1,1,1
Legionella,2,2,2,2
No Finding,2,2,2,2
Pneumocystis,9,8,12,12
SARS,16,16,16,16


In [9]:
WIDTH = 256
HEIGHT = 256

In [10]:
transformation = transforms.Compose([
    transforms.Resize((WIDTH, HEIGHT)),
    transforms.Grayscale(),
    transforms.ColorJitter(
        brightness=0.4,
        contrast=0.4,
        saturation=0.4,
    ),
    transforms.RandomRotation(10),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225]),
    transforms.ToTensor()
])

In [11]:
path = Path(os.getcwd())
dataset_path = path / 'dataset'
os.mkdir('dataset') if not dataset_path.is_dir() else None   

In [12]:
def copy(old_file_name,old_file_dir_path, new_file_name, new_file_dir_path):
    old_file_location = old_file_dir_path / old_file_name
    if old_file_location.is_file():
        if new_file_dir_path.is_dir():
            with open(old_file_location, 'rb') as c:
                data = c.read()
                with open(new_file_dir_path/new_file_name, 'wb') as p:
                    p.write(data)
        else:
            raise FileNotFoundError(new_file_dir_path)
    else:
        raise FileNotFoundError(old_file_location)

In [13]:
images_dir = repo_path / 'images'
print(images_dir, images_dir.is_dir())

/home/ash/Desktop/covid-19-detection/covid-chestxray-dataset/images True


In [14]:
for disease in illness:
    for x,data in enumerate(final_data[final_data.finding == disease].values):
        sex, age,_, image_name, _ = data 
        fileType = image_name.split('.')[-1]
        temp_path = dataset_path / disease.replace(', ','_')
        os.mkdir(temp_path) if not temp_path.is_dir() else None
        copy(image_name, images_dir,f"{x}__{sex}__{age}.{fileType}",temp_path)

In [15]:
max_item = 0
for item in os.listdir(dataset_path):
    size = len(os.listdir(dataset_path / item))
    max_item = size if size > max_item else max_item
    print(f"{item:<20}{size}")

Chlamydophila       1
Pneumocystis        12
COVID-19_ARDS       11
ARDS                4
No Finding          2
SARS                16
Streptococcus       13
COVID-19            144
Klebsiella          1
Legionella          2


In [18]:
from Net import CustomNet

In [20]:
model = CustomNet(3)